In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import spacy


In [2]:
df = pd.read_csv("diario_emociones.csv")

# Split inicial estratificado (80% entrenamiento, 20% test)
X_temp, X_test, y_temp, y_test = train_test_split(
    df["texto"],        # Features: solo columna de texto
    df["emocion"],      # Target: emociones
    test_size=0.2,
    stratify=df["emocion"],
    random_state=42
)

# Crear DataFrame para balanceo
train_df = pd.DataFrame({
    "texto": X_temp,
    "emocion": y_temp
})


In [3]:
balanced_dfs = []
for emocion in train_df["emocion"].unique():
    subset = train_df[train_df["emocion"] == emocion]
    balanced_dfs.append(
        resample(subset,
               replace=True,
               n_samples=500,
               random_state=42)
    )
    
train_df_balanced = pd.concat(balanced_dfs, ignore_index=True)

# Separar features y target balanceados
X_train = train_df_balanced["texto"]
y_train = train_df_balanced["emocion"]


In [4]:
nlp = spacy.load("es_core_news_sm")

def limpiar_texto(texto):
    doc = nlp(texto.lower())
    tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
            lemma = token.lemma_.strip()
            # Manejar signos importantes
            if lemma in ["¡", "!"]:
                tokens.append("EXCLAMACION")
            elif lemma in ["¿", "?"]:
                tokens.append("PREGUNTA")
            else:
                tokens.append(lemma)
    return " ".join(tokens)

# Aplicar limpieza
X_train_limpio = X_train.apply(limpiar_texto)
X_test_limpio = X_test.apply(limpiar_texto)

In [5]:
vectorizer = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.75,
    stop_words=["de", "la", "que", "y", "en"]  # Añade tus stopwords
)

X_train_vec = vectorizer.fit_transform(X_train_limpio)
X_test_vec = vectorizer.transform(X_test_limpio)

In [6]:
normalizer = Normalizer(norm='l2')
X_train = normalizer.fit_transform(X_train_vec.toarray())
X_test = normalizer.transform(X_test_vec.toarray())

In [7]:
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)  # Usamos y_test original


In [8]:
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, 
    y_train_enc,
    test_size=0.2,
    stratify=y_train_enc,
    random_state=42
)


In [10]:
model = Sequential()
model.add(Dense(
    512,
    activation='relu',
    input_shape=(X_train.shape[1],),
    kernel_regularizer=l2(0.005)
))
model.add(Dropout(0.4))
model.add(Dense(
    256,
    activation='relu',
    kernel_regularizer=l2(0.003))
)
model.add(Dropout(0.3))
model.add(Dense(
    len(label_encoder.classes_),
    activation='softmax'
))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

C:\Users\walte\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    min_delta=0.001
)

history = model.fit(
    X_train_final, y_train_final,
    batch_size=128,
    epochs=200,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=2
)


Epoch 1/200
16/16 - 2s - 147ms/step - accuracy: 0.6265 - loss: 4.5876 - val_accuracy: 0.9780 - val_loss: 3.5250
Epoch 2/200
16/16 - 0s - 10ms/step - accuracy: 0.9855 - loss: 2.7425 - val_accuracy: 1.0000 - val_loss: 1.8157
Epoch 3/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 1.3211 - val_accuracy: 1.0000 - val_loss: 0.9437
Epoch 4/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.7874 - val_accuracy: 1.0000 - val_loss: 0.6123
Epoch 5/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.5238 - val_accuracy: 1.0000 - val_loss: 0.4317
Epoch 6/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.3946 - val_accuracy: 1.0000 - val_loss: 0.3429
Epoch 7/200
16/16 - 0s - 13ms/step - accuracy: 1.0000 - loss: 0.3258 - val_accuracy: 1.0000 - val_loss: 0.2950
Epoch 8/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.2867 - val_accuracy: 1.0000 - val_loss: 0.2644
Epoch 9/200
16/16 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.2615 - val_accuracy: 1.0000 - val_loss: 0.2446


In [17]:
def predecir_emocion(texto, umbrales_clase=None):
    # Limpieza
    texto_limpio = limpiar_texto(texto)
    
    # Vectorización
    texto_vec = vectorizer.transform([texto_limpio])
    
    # Normalización
    texto_norm = normalizer.transform(texto_vec.toarray())
    
    # Predicción
    prediccion = model.predict(texto_norm, verbose=0)[0]
    clase_predicha = np.argmax(prediccion)
    confianza = prediccion[clase_predicha]
    
    # Umbrales dinámicos (ejemplo)
    if umbrales_clase is None:
        umbrales_clase = {
            "tristeza": 0.6,
            "ansiedad": 0.65,
            "estrés": 0.55,
            "enojo": 0.7,
            "felicidad": 0.6
        }
    
    nombre_clase = label_encoder.inverse_transform([clase_predicha])[0]
    umbral = umbrales_clase.get(nombre_clase, 0.65)
    
    return (nombre_clase, confianza) if confianza >= umbral else ("indefinido", confianza)

In [19]:
def chatbot_respuesta(texto):
    emocion, confianza = predecir_emocion(texto)
    
    recursos = {
        "tristeza": {
            "mensaje": "🔵 Pareces sentir tristeza. Te recomiendo:",
            "acciones": [
                "Meditación guiada 'Alivio emocional' (10 min)",
                "Ejercicio: Escribe una carta para liberar emociones"
            ]
        },
        "ansiedad": {
            "mensaje": "🟠 Detecto señales de ansiedad. Prueba:",
            "acciones": [
                "Técnica de respiración 4-7-8 (instrucciones)",
                "Ejercicio de grounding: 5-4-3-2-1"
            ]
        },
        "estrés": {
            "mensaje": "🟢 Sugerencias para manejar el estrés:",
            "acciones": [
                "Meditación 'Libera tensiones' (15 min)",
                "Prioriza tareas con matriz Eisenhower"
            ]
        },
        "enojo": {
            "mensaje": "🔴 Detecto frustración. Intenta:",
            "acciones": [
                "Ejercicio físico de alta intensidad",
                "Técnica de pausa consciente de 5 minutos"
            ]
        },
        "felicidad": {
            "mensaje": "🟡 ¡Me alegra verte así! Mantén esto:",
            "acciones": [
                "Registra este momento en tu diario positivo",
                "Comparte tu estado con alguien especial"
            ]
        },
        "indefinido": {
            "mensaje": "⚪️ Necesito entenderte mejor. ¿Podrías:",
            "acciones": [
                "Describir cómo te sientes con más detalle?",
                "Contarme qué ha pasado recientemente?"
            ]
        }
    }
    
    respuesta = recursos.get(emocion, recursos["indefinido"])
    return (
        f"{respuesta['mensaje']}\n" + 
        "\n".join([f"- {accion}" for accion in respuesta['acciones']]) + 
        f"\n\nConfianza del modelo: {confianza:.2%}"
    )


In [21]:
test_cases = [
    "No tengo energía para nada hoy",
    "¡Acabo de conseguir el trabajo de mis sueños!",
    "Estoy harto de las injusticias en el trabajo",
    "Me siento mareado y con dificultad para respirar",
    "No sé qué me pasa, todo me parece gris"
]

for caso in test_cases:
    print(f"🧑 Usuario: {caso}")
    print(f"🤖 Chatbot:\n{chatbot_respuesta(caso)}\n")
    print("―" * 50)

🧑 Usuario: No tengo energía para nada hoy
🤖 Chatbot:
⚪️ Necesito entenderte mejor. ¿Podrías:
- Describir cómo te sientes con más detalle?
- Contarme qué ha pasado recientemente?

Confianza del modelo: 47.55%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: ¡Acabo de conseguir el trabajo de mis sueños!
🤖 Chatbot:
🟢 Sugerencias para manejar el estrés:
- Meditación 'Libera tensiones' (15 min)
- Prioriza tareas con matriz Eisenhower

Confianza del modelo: 68.00%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: Estoy harto de las injusticias en el trabajo
🤖 Chatbot:
🟢 Sugerencias para manejar el estrés:
- Meditación 'Libera tensiones' (15 min)
- Prioriza tareas con matriz Eisenhower

Confianza del modelo: 68.00%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: Me siento mareado y con dificultad para respirar
🤖 Chatbot:
🔵 Pareces sentir tristeza. Te recomiendo:
- Meditación guiada 'Alivio emocional' (10 min)
- Ejercicio: Escribe una carta para libera